### Sample script for user-based collaborative filtering  

#### Import libraries

In [28]:
import numpy as np
import pandas as pd

#### Parameters  

In [29]:
csv_in = 'sushi3b.5000.10.score.csv'
# min number of common items between target user and users in DB
min_common_items = 4
# min number of users who evaluated an item to be recommended
min_eval_users = 3

# To show more rows and columns
pd.options.display.max_rows = 999 
pd.options.display.max_columns = 999 

#### Read CSV file  

In [30]:
df = pd.read_csv(csv_in, delimiter=',', skiprows=0, header=0)
print(df.shape)
display(df.info())
display(df.head())

(5000, 100)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 100 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   ebi             5000 non-null   int64
 1   anago           5000 non-null   int64
 2   maguro          5000 non-null   int64
 3   ika             5000 non-null   int64
 4   uni             5000 non-null   int64
 5   tako            5000 non-null   int64
 6   ikura           5000 non-null   int64
 7   tamago          5000 non-null   int64
 8   toro            5000 non-null   int64
 9   amaebi          5000 non-null   int64
 10  hotategai       5000 non-null   int64
 11  tai             5000 non-null   int64
 12  akagai          5000 non-null   int64
 13  hamachi         5000 non-null   int64
 14  awabi           5000 non-null   int64
 15  samon           5000 non-null   int64
 16  kazunoko        5000 non-null   int64
 17  shako           5000 non-null   int64
 18  saba           

None

,ebi,anago,maguro,ika,uni,tako,ikura,tamago,toro,amaebi,hotategai,tai,akagai,hamachi,awabi,samon,kazunoko,shako,saba,chu_toro,hirame,aji,kani,kohada,torigai,unagi,tekka_maki,kanpachi,mirugai,kappa_maki,geso,katsuo,iwashi,hokkigai,shimaaji,kanimiso,engawa,negi_toro,nattou_maki,sayori,takuwan_maki,botanebi,tobiko,inari,mentaiko,sarada,suzuki,tarabagani,ume_shiso_maki,komochi_konbu,tarako,sazae,aoyagi,toro_samon,sanma,hamo,nasu,shirauo,nattou,ankimo,kanpyo_maki,negi_toro_maki,gyusashi,hamaguri,basashi,fugu,tsubugai,ana_kyu_maki,hiragai,okura,ume_maki,sarada_maki,mentaiko_maki,buri,shiso_maki,ika_nattou,zuke,himo,kaiware,kurumaebi,mekabu,kue,sawara,sasami,kujira,kamo,himo_kyu_maki,tobiuo,ishigakidai,mamakari,hoya,battera,kyabia,karasumi,uni_kurage,karei,hiramasa,namako,shishamo,kaki
0,-1,0,-1,4,2,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,4,-1,2,-1,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,-1,0,-1,1,-1,-1,-1,0,-1,-1,-1,-1,0,-1,-1,-1,1,2,-1,0,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,3,4,-1,-1,-1,3,-1,-1,-1,-1,-1,-1,4,-1,4,-1,-1,-1,-1,-1,-1,-1,-1,-1,3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,-1,-1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,4,-1,-1,3,4,1,-1,-1,4,3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,3,-1,-1,-1,-1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,-1,-1,3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,1,-1,-1,-1,-1,4,-1,-1,-1,0,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,4,-1,-1,3,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0


#### Replace -1 into NaN (for not to be used during calculation)  

In [31]:
df = df.replace(-1, np.nan)
display(df.head())

,ebi,anago,maguro,ika,uni,tako,ikura,tamago,toro,amaebi,hotategai,tai,akagai,hamachi,awabi,samon,kazunoko,shako,saba,chu_toro,hirame,aji,kani,kohada,torigai,unagi,tekka_maki,kanpachi,mirugai,kappa_maki,geso,katsuo,iwashi,hokkigai,shimaaji,kanimiso,engawa,negi_toro,nattou_maki,sayori,takuwan_maki,botanebi,tobiko,inari,mentaiko,sarada,suzuki,tarabagani,ume_shiso_maki,komochi_konbu,tarako,sazae,aoyagi,toro_samon,sanma,hamo,nasu,shirauo,nattou,ankimo,kanpyo_maki,negi_toro_maki,gyusashi,hamaguri,basashi,fugu,tsubugai,ana_kyu_maki,hiragai,okura,ume_maki,sarada_maki,mentaiko_maki,buri,shiso_maki,ika_nattou,zuke,himo,kaiware,kurumaebi,mekabu,kue,sawara,sasami,kujira,kamo,himo_kyu_maki,tobiuo,ishigakidai,mamakari,hoya,battera,kyabia,karasumi,uni_kurage,karei,hiramasa,namako,shishamo,kaki
0,NaN,0.0,NaN,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,2.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,3.0,4.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,3.0,4.0,1.0,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


#### Return Series of similiarity of users  
by using Pearson correlation coefficient (-1 .. 1).  

In [32]:
def get_sim_ser_by_pearson(df_users, ser_target):
    corr_all = df_users.corrwith(ser_target, axis=1).dropna()
    return corr_all

In [33]:
similarity = 'pearson2'
def get_sim_ser_by_pearson2(df_users, ser_target):
    c = df_users.corrwith(ser_target, axis=1).dropna()
    return c if similarity == 'pearson' else (c+1)/2

#### Prediction of scores for the target user  
(for items with no score)  
based on deviation from average  

In [34]:
def predict_scores(df_sim, ser_sim, ser_target):
    ret = {}
    df_sim_ave = df_sim.mean(axis=1)
    #print(df_sim.head())  # debug
    target_ave = ser_target.mean()
    #print(target_ave)  # debug
    for item1 in df_sim.columns:
        if item1 in ser_target.index: continue
        v1 = df_sim[item1]
        if v1.notnull().sum() < min_eval_users: continue
        v1 -= df_sim_ave
        v11 = v1[ v1.notnull() ]
        t11 = ser_sim[ v1.notnull() ]
        pred1 = target_ave + (v11 * t11).sum() / np.abs(t11).sum()
        ret[item1] = pred1
    
    ser_ret = pd.Series(ret)
    
    return ser_ret.sort_values(ascending=False)

#### Function for user-based collaborative filtering.  

arguments: dictionary of scores for the target user  

ex)
get_recomm_by_user_sim({'maguro':1, 'ika':1, 'uni':3,
                        'awabi':4, 'hirame':4, 'aoyagi':4})  
-> return list such as [('akagai', 2.9835603009918303), ('mirugai', 2.945676429588114), ...]

In [35]:
def get_recomm_by_user_sim(df, target_dic):
    ser_target = pd.Series(target_dic)
    # make dataframe with columns included in target_dic
    df_scores = df[ ser_target.index ]
    #display(df_scores)  # debug
    n_same_items = df_scores.notnull().sum(axis=1)
    #display(df_scores.notnull())  # debug
    df_common = df_scores[ n_same_items>=min_common_items ]
    #display(df_common)  # debug
    
    ser_sim = get_sim_ser_by_pearson2(df_common, ser_target)    
    #print(ser_sim)  # debug
    df_sim = df.loc[ ser_sim.index ]
    #display(df_sim)  # debug
    recomm = predict_scores(df_sim, ser_sim, ser_target)

    return recomm

#### Do recommendation  

Number of items calculated: 46
Recommendation:
ankimo     4.176725
kohada     3.716200
mirugai    3.598203
shako      3.581244
akagai     3.539989
dtype: float64
Number of items calculated: 73
Recommendation:
toro_samon    3.993705
okura         3.607180
negi_toro     3.338903
tarabagani    3.312345
kanpachi      3.228670
dtype: float64

In [36]:
similarity = 'pearson'
recomm = get_recomm_by_user_sim(df,
                                {'maguro':1, 'ika':1, 'uni':3,
                                 'awabi':4, 'hirame':4, 'aoyagi':4})
print('Number of items calculated:', len(recomm))
print('Recommendation:')
print(recomm.head())

recomm = get_recomm_by_user_sim(df,
                                {'anago': 3, 'maguro': 4, 'ikura': 3,
                                 'hamachi': 4, 'samon': 4, 'unagi': 3,
                                 'suzuki': 2, 'hamo': 1, 'nasu': 1,
                                 'shiso_maki': 2})
print('Number of items calculated:', len(recomm))
print('Recommendation:')
print(recomm.head())

Number of items calculated: 46
Recommendation:
ankimo     4.176725
kohada     3.716200
mirugai    3.598203
shako      3.581244
akagai     3.539989
dtype: float64
Number of items calculated: 73
Recommendation:
toro_samon    3.993705
okura         3.607180
negi_toro     3.338903
tarabagani    3.312345
kanpachi      3.228670
dtype: float64


In [37]:
similarity = 'pearson2'
recomm = get_recomm_by_user_sim(df,
                                {'maguro':1, 'ika':1, 'uni':3,
                                 'awabi':4, 'hirame':4, 'aoyagi':4})
print('Number of items calculated:', len(recomm))
print('Recommendation:')
print(recomm.head())

recomm = get_recomm_by_user_sim(df,
                                {'anago': 3, 'maguro': 4, 'ikura': 3,
                                 'hamachi': 4, 'samon': 4, 'unagi': 3,
                                 'suzuki': 2, 'hamo': 1, 'nasu': 1,
                                 'shiso_maki': 2})
print('Number of items calculated:', len(recomm))
print('Recommendation:')
print(recomm.head())

Number of items calculated: 46
Recommendation:
toro_samon    3.933693
kurumaebi     3.715042
hotategai     3.629251
chu_toro      3.628787
amaebi        3.610793
dtype: float64
Number of items calculated: 73
Recommendation:
toro_samon    3.707632
negi_toro     3.586865
toro          3.530291
tarabagani    3.343768
zuke          3.327067
dtype: float64
